# Trump or Troll
## Final Project: Shivam Jindal (sj2546) and Mayank Vanjani (mv1506)

Classify tweets sent by President Trump versus Russian trolls

In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, MiniBatchKMeans


In [7]:

df = pd.read_excel("TrumpTwitterAll.xlsx")
df2 = pd.read_csv("IRAhandle_tweets_1.txt")


trump_tweets_df is a dataframe containing all of the trump tweets from our dataset  
troll_tweets_df is a dataframe containing all of the troll tweets from our dataset

In [31]:
#df.head(5)
print(df.shape)
print(df2.shape)
print(df.columns)
print(df2.columns)

trump_tweets_df = df["Tweet"]
troll_tweets_df = df2["content"][:30078]

(30078, 5)
(243891, 21)
Index(['Date', 'Time', 'Tweet', 'Client', 'Client Simplified'], dtype='object')
Index(['external_author_id', 'author', 'content', 'region', 'language',
       'publish_date', 'harvested_date', 'following', 'followers', 'updates',
       'post_type', 'account_type', 'retweet', 'account_category',
       'new_june_2018', 'alt_external_id', 'tweet_id', 'article_url',
       'tco1_step1', 'tco2_step1', 'tco3_step1'],
      dtype='object')
(30078,)


In [10]:
trump_tweets_df.head(5)

<class 'pandas.core.series.Series'>


In [5]:
troll_tweets_df.head(5)

0    "We have a sitting Democrat US Senator on tria...
1    Marshawn Lynch arrives to game in anti-Trump s...
2    Daughter of fallen Navy Sailor delivers powerf...
3    JUST IN: President Trump dedicates Presidents ...
4    19,000 RESPECTING our National Anthem! #StandF...
Name: content, dtype: object

In [32]:
vectorizer = TfidfVectorizer( 
                max_df=0.5, # max doc freq (as a fraction) of any word to include in the vocabulary
                min_df=2,   # min doc freq (as doc counts) of any word to include in the vocabulary
                max_features=10000,           # max number of words in the vocabulary
                stop_words='english',         # remove English stopwords
                use_idf=True )        # use IDF scores
all_my_tweeties = trump_tweets_df.append(troll_tweets_df)
all_my_tweeties.head(5)

X = vectorizer.fit_transform(all_my_tweeties)


In [52]:
pd.options.display.max_colwidth = 2000

print(all_my_tweeties[1000])

1000    Thank you American Legion Post 610- for hosting @Mike_Pence & I for a roundtable with labor leaders. #LaborDay #MAGA https://t.co/r0cwJlV38L
1000                                Buona domenica con la #buonalettura geopolitica della settimana  https://t.co/X17mpXcPfq https://t.co/YaZwFl0boc
dtype: object


In [53]:
doc_ind = 1000  # Index of an example document
xi = X[doc_ind,:].todense()
term_ind = xi.argsort()[:, ::-1]
xi_sort = xi[0,term_ind]
terms = vectorizer.get_feature_names()

for i in range(10):
    term = terms[term_ind[0,i]]
    tfidf = xi[0,term_ind[0,i]]
    print('{0:20s} {1:f} '.format(term, tfidf))

labor                0.428762 
mike_pence           0.412798 
hosting              0.394354 
maga                 0.355841 
leaders              0.342455 
post                 0.329780 
american             0.284062 
thank                0.217162 
https                0.098706 
giulio               0.000000 


In [59]:
km = KMeans(n_clusters=2, init='k-means++', n_init=1)

In [60]:
km.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=2, n_init=1, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [62]:
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
for i in range(2):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :20]:
        print(' %s' % terms[ind], end='')
    print()

Cluster 0: https http trump great thank just people blacklivesmatter new enlist на like obama america don make di good time donald
Cluster 1: realdonaldtrump thanks president great trump run donald thank mr love vote 2016 trump2016 need country true america best good make


In [86]:
first_half = np.zeros(30078)
second_half = first_half+1

labels = np.append(first_half, second_half)

In [87]:
km.labels_


array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

In [90]:
labelkm = km.labels_
from sklearn.metrics import confusion_matrix
C = confusion_matrix(labels,labelkm)

Csum = np.sum(C, axis=0)
Cnorm = C / Csum[None,:]
print(Cnorm)

[[0.40696152 0.98182005]
 [0.59303848 0.01817995]]
